In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42


In [2]:
dataset = 'model/keypoint.csv'
model_save_path = 'model/keypoint_classifier.hdf5'
NUM_CLASSES = 3

In [3]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [20]:
X_train[0]

array([ 0.        ,  0.        , -0.2556391 , -0.04511278, -0.45864663,
       -0.22556391, -0.518797  , -0.45112783, -0.41353384, -0.60902256,
       -0.38345864, -0.6466165 , -0.43609023, -0.91729325, -0.35338345,
       -0.96240604, -0.2706767 , -0.86466163, -0.18796992, -0.66917294,
       -0.17293233, -1.        , -0.08270676, -0.93233085, -0.04511278,
       -0.7518797 , -0.01503759, -0.62406015,  0.03759398, -0.86466163,
        0.07518797, -0.7218045 ,  0.06766918, -0.5338346 ,  0.15037593,
       -0.5338346 ,  0.19548872, -0.65413535,  0.19548872, -0.5413534 ,
        0.15789473, -0.41353384], dtype=float32)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [10]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 42)                0         
                                                                 
 dense_5 (Dense)             (None, 20)                860       
                                                                 
 dropout_5 (Dropout)         (None, 20)                0         
                                                                 
 dense_6 (Dense)             (None, 10)                210       
                                                                 
 dense_7 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________


In [11]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)

es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [12]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/27 [>.............................] - ETA: 22s - loss: 1.2769 - accuracy: 0.3516
Epoch 1: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 1s 12ms/step - loss: 1.1796 - accuracy: 0.3542 - val_loss: 1.0458 - val_accuracy: 0.5337
Epoch 2/1000
25/27 [==========================>...] - ETA: 0s - loss: 1.0917 - accuracy: 0.3931
Epoch 2: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 0s 5ms/step - loss: 1.0912 - accuracy: 0.3932 - val_loss: 1.0107 - val_accuracy: 0.6137
Epoch 3/1000
22/27 [=======================>......] - ETA: 0s - loss: 1.0561 - accuracy: 0.4169
Epoch 3: saving model to model\keypoint_classifier.hdf5
27/27 [==============================] - 0s 5ms/step - loss: 1.0519 - accuracy: 0.4186 - val_loss: 0.9767 - val_accuracy: 0.6253
Epoch 4/1000
18/27 [===================>..........] - ETA: 0s - loss: 1.0224 - accuracy: 0.4553
Epoch 4: saving model to model\keypoint_classifier.hdf5
27/

In [14]:
model = tf.keras.models.load_model(model_save_path)